In the context of this problem, what is a false positive?  
the 7 where cat was predicted but was actually a dog

In the context of this problem, what is a false negative?  
the 13 where dog was predicted but was actually a cat

How would you describe this model?  
accurate well over half the time


In [47]:
tp = 34
tn = 46
fp = 7
fn = 13


In [48]:
accuracy = (tp + tn) / (tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp+fn)

In [49]:
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import os
import prepare
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


In [50]:
cody_df = pd.read_csv('c3.csv')
cody_df

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect
...,...,...,...,...
195,No Defect,No Defect,Defect,Defect
196,Defect,Defect,No Defect,No Defect
197,No Defect,No Defect,No Defect,No Defect
198,No Defect,No Defect,Defect,Defect


An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

optimize for true positives and allow for more false positives

In [51]:
c_subset = cody_df[cody_df.actual == 'Defect']
c_subset
sens1= (c_subset.model1 == c_subset.actual).mean()
sens2= (c_subset.model2 == c_subset.actual).mean()
sens3= (c_subset.model3 == c_subset.actual).mean()
print(sens1, sens2, sens3)

#model 3 offers the best sensitivity for actual defects 

0.5 0.5625 0.8125


recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

optimize for true negatives and allow for more false negatives

In [52]:
c_subset2 = cody_df[cody_df.actual == 'No Defect']

spec1= (c_subset2.model1 == c_subset2.actual).mean()
spec2= (c_subset2.model2 == c_subset2.actual).mean()
spec3= (c_subset2.model3 == c_subset2.actual).mean()
print(spec1, spec2, spec3)

#model 1 has the highest sensitivity for detecting true 'no defects'

0.9891304347826086 0.5597826086956522 0.532608695652174


You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

In [53]:
gup_df = pd.read_csv('gives_you_paws.csv')
gup_df

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog
...,...,...,...,...,...
4995,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog
4997,dog,cat,cat,dog,dog
4998,cat,cat,cat,cat,dog


Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [54]:
gup_df.actual.value_counts()



dog    3254
cat    1746
Name: actual, dtype: int64

In [55]:
gup_df['baseline'] = 'dog'
gup_df

,actual,model1,model2,model3,model4,baseline
0,cat,cat,dog,cat,dog,dog
1,dog,dog,cat,cat,dog,dog
2,dog,cat,cat,cat,dog,dog
3,dog,dog,dog,cat,dog,dog
4,cat,cat,cat,dog,dog,dog
...,...,...,...,...,...,...
4995,dog,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog,dog
4997,dog,cat,cat,dog,dog,dog
4998,cat,cat,cat,cat,dog,dog


In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [56]:
acu_mod1 = (gup_df.actual == gup_df.model1).mean()
acu_mod2 = (gup_df.actual == gup_df.model2).mean()
acu_mod3 = (gup_df.actual == gup_df.model3).mean()
acu_mod4 = (gup_df.actual == gup_df.model4).mean()
acu_base = (gup_df.actual == gup_df.baseline).mean()
print(acu_mod1, acu_mod2, acu_mod3, acu_mod4, acu_base)

0.8074 0.6304 0.5096 0.7426 0.6508


accuracy-wise model1 and model4 outperform the baseline with model1 performing the best.

Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?

In [58]:
gup_subset1 = gup_df[gup_df.model1 == 'dog']
gup_subset2 = gup_df[gup_df.model2 == 'dog']
gup_subset3 = gup_df[gup_df.model3 == 'dog']
gup_subset4 = gup_df[gup_df.model4 == 'dog']
(gup_subset1.actual == gup_subset1.model1).mean()

0.8900238338440586

In [59]:
(gup_subset2.actual == gup_subset2.model2).mean()

0.8931767337807607

In [60]:
(gup_subset3.actual == gup_subset3.model3).mean()

0.6598883572567783

In [61]:
(gup_subset4.actual == gup_subset4.model4).mean()

# model 2 appears to be the most sensitive to true positives for dog pictures

0.7312485304490948

Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

In [63]:
gup_subset1 = gup_df[gup_df.model1 == 'cat']
gup_subset2 = gup_df[gup_df.model2 == 'cat']
gup_subset3 = gup_df[gup_df.model3 == 'cat']
gup_subset4 = gup_df[gup_df.model4 == 'cat']

(gup_subset1.actual == gup_subset1.model1).mean()

0.6897721764420747

In [64]:
(gup_subset2.actual == gup_subset2.model2).mean()

0.4841220423412204

In [65]:
(gup_subset3.actual == gup_subset3.model3).mean()

0.358346709470305

In [66]:
(gup_subset4.actual == gup_subset4.model4).mean()

0.8072289156626506

In [ ]:
# model 2 appears to be the most sensitive to true positives for cat pictures

Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

In [46]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [69]:
pd.DataFrame(classification_report(gup_df.actual, gup_df.model1, labels = ['cat', 'dog'], output_dict = True))

,cat,dog,accuracy,macro avg,weighted avg
precision,0.689772,0.890024,0.8074,0.789898,0.820096
recall,0.815006,0.803319,0.8074,0.809162,0.807400
f1-score,0.747178,0.844452,0.8074,0.795815,0.810484
support,1746.000000,3254.000000,0.8074,5000.000000,5000.000000
